In [ ]:
"""
  Se importan las libreriías necesarias, el conjunto de audios
  ESC-50 y los archivos para la red neuronal de YAMNet
"""
!git clone https://github.com/tensorflow/models.git
!cd models/research/audioset/yamnet
!curl -O https://storage.googleapis.com/audioset/yamnet.h5
!python yamnet_test.py
!git clone https://github.com/karoldvl/ESC-50.git
!cp -r models/research/audioset/yamnet/* .
import csv
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from yamnet_test import *
from features import *
import params as yamnet_params
import yamnet as yamnet_model
import pandas as pd
import os
model = hub.load('https://tfhub.dev/google/yamnet/1')

In [22]:
"""
  Inicialmente, se importa el dataframe del conjunto ESC-50 para obteneer los
  nombres de los audios y poder procesarlos con YAMNet
"""
metadatos_df = pd.read_csv("ESC-50/meta/esc50.csv")
names = metadatos_df.filename
last = metadatos_df.fold.iloc [-1]+1
"""
  La función "yamnet_caracteristicas", procesa todos los audios del conjunto
  de forma sucesiva, haciendo la media y la varianza de cada uno, para 
  obtener una lista de 2000x2048 (2000 audios y 1024 la media de todas la 
  características de los 5 segundos + 1024 la varianza de todas la 
  características de los 5 segundos )
"""

def yamnet_caracteristicas(names):
  lista = [] 

  for i in range(len(names)):
    target = names[i]
    path = "/content/ESC-50/audio/"+target
    wav_file_name = path
    sample_rate, wav_data = wavfile.read(wav_file_name, 'rb')
    waveform = wav_data / tf.int16.max
    scores, embeddings, spectrogram = model(waveform)
    mean_embeddings = np.mean(embeddings, axis=0)
    var_embeddings = np.var(embeddings, axis=0)
    c = []
    c.extend(mean_embeddings)
    c.extend(var_embeddings)
    lista.append(c)
    print(len(lista),end=" ")
    if len(lista) % 20 == 0:
      print("")

  return np.array(lista)

In [3]:
lista = yamnet_caracteristicas(names)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 
41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 
61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 
81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 
101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 
121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 
141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 
161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 
181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 
201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 
221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 
241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 
261 262 263 264 265 266 267 268 269 270 271 272 273 274

In [32]:
"""
  La función "dataframe" transforma la lista con las características en una 
  tabla organizada de forma, que se pueda dividir de forma sencilla para la 
  validación cruzada y la clasificación
"""

def dataframe(datos):
  atributos = list(map(lambda x:"Att"+str(x),range(2048)))
  metadatos_df = pd.read_csv("ESC-50/meta/esc50.csv")
  metadatos_df ["Clase"] = (metadatos_df.category == "coughing").astype(int)
  Caracteristicas_df = pd.DataFrame(datos,metadatos_df.filename,atributos)
  metadatos_df = metadatos_df.set_index("filename")
  Caracteristicas_df["fold"] = metadatos_df["fold"]
  Caracteristicas_df["Clase"] = metadatos_df["Clase"]
  Caracteristicas_df.to_csv('características(yamnet).csv')
  return Caracteristicas_df
dataframe(lista)



,Att0,Att1,Att2,Att3,Att4,Att5,Att6,Att7,Att8,Att9,Att10,Att11,Att12,Att13,Att14,Att15,Att16,Att17,Att18,Att19,Att20,Att21,Att22,Att23,Att24,Att25,Att26,Att27,Att28,Att29,Att30,Att31,Att32,Att33,Att34,Att35,Att36,Att37,Att38,Att39,...,Att2010,Att2011,Att2012,Att2013,Att2014,Att2015,Att2016,Att2017,Att2018,Att2019,Att2020,Att2021,Att2022,Att2023,Att2024,Att2025,Att2026,Att2027,Att2028,Att2029,Att2030,Att2031,Att2032,Att2033,Att2034,Att2035,Att2036,Att2037,Att2038,Att2039,Att2040,Att2041,Att2042,Att2043,Att2044,Att2045,Att2046,Att2047,fold,Clase
filename,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1-100032-A-0.wav,1.779540,0.403408,0.577516,0.064785,2.924030,2.498055,0.000000,2.081190,2.686362,1.240811,2.100662,3.650690,0.131920,2.008808,2.390445,0.769741,2.377091,1.070852,1.798648,1.699093,0.190135,3.731860,0.502470,0.397720,1.752992,0.272637,1.324439,3.057528,2.227383,0.073194,0.556813,1.244318,2.657946,0.617994,0.351683,1.562894,1.873854,2.917097,0.551681,0.459555,...,0.005686,0.223729,0.002114,0.056331,0.005917,0.007362,2.306438,1.211878,0.654683,0.005899,0.000000,1.923445,0.121309,0.002674,1.398462,0.743628,0.263034,1.683293,0.294125,0.001578,8.796892e-02,0.069234,0.048821,0.879904,0.032505,0.787748,0.874781,0.031108,0.098056,1.395155,1.943279,1.313113e-02,2.827352,0.702822,0.144989,0.009785,0.564423,1.549536,1,0
1-100038-A-14.wav,0.000000,0.148130,0.912608,0.012334,0.000000,0.000000,0.000000,0.000000,0.000000,0.012466,0.000000,0.000000,0.001692,0.000000,0.000000,0.000659,0.000000,0.001872,0.000000,0.000000,0.000000,0.000000,0.000000,0.023297,0.000245,0.001176,0.005234,0.000000,0.000000,0.000324,0.000000,0.000000,0.000000,0.000000,0.268368,0.000000,0.000000,0.000000,0.000000,0.130666,...,0.000000,0.000007,0.028571,0.133534,0.322856,0.000000,0.000000,0.000000,0.000000,0.044704,0.000138,0.000000,0.292622,0.035640,0.000000,0.000000,0.074506,0.000000,0.021493,0.004372,8.926112e-07,0.000000,0.348272,0.000000,0.000000,0.000000,0.000000,0.002000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.002557,0.026081,0.000000,0.000000,1,0
1-100210-A-36.wav,0.000000,0.000000,0.113216,1.652952,0.000000,0.000000,0.000000,0.003575,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038183,0.000000,0.021315,0.000000,0.000000,0.111713,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.351624,0.000000,0.000000,0.228470,0.034467,0.000000,1.428239,0.000000,0.000000,0.000000,...,0.016334,0.000000,0.000000,0.000000,0.060307,0.000000,0.000000,0.000000,0.000000,0.000000,0.109925,0.000000,0.000000,0.321784,0.000000,0.000000,0.170435,0.000000,0.000000,0.000000,0.000000e+00,0.001891,0.003983,0.000000,0.000000,0.000000,0.000000,0.005748,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.101207,0.530641,0.000000,0.000000,1,0
1-100210-B-36.wav,0.000000,0.008155,0.111582,1.656615,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050973,0.000000,0.010635,0.000000,0.000000,0.271945,0.000000,0.000000,0.000000,0.000000,0.000000,0.000438,0.003284,0.000000,0.000000,1.135686,0.000000,0.000000,0.296453,0.195558,0.000000,1.388892,0.000000,0.000000,0.000000,...,0.000019,0.000000,0.000000,0.000000,0.048467,0.000000,0.000000,0.000000,0.000000,0.000000,0.077821,0.000000,0.000000,0.201588,0.000000,0.000000,0.130035,0.000000,0.000000,0.000000,0.000000e+00,0.026542,0.000116,0.000000,0.000000,0.000000,0.000000,0.007462,0.000000,0.000000,0.000000,4.981066e-08,0.000000,0.000000,0.012354,0.817915,0.000000,0.000000,1,0
1-101296-A-19.wav,0.000841,0.039994,0.026224,0.261450,0.000745,0.000000,0.000000,0.000814,0.000000,0.000000,0.000000,0.000000,0.022463,0.000605,0.000000,0.008603,0.000000,0.004721,0.009393,0.000000,0.008715,0.000000,0.002755,0.092396,0.000742,0.000438,0.002470,0.048356,0.000000,0.223862,0.031029,0.000000,0.000000,0.000000,0.429355,0.000000,0.000850,0.000000,0.000000,0.009423,...,0.000075,0.000000,0.000000,0.091148